In [1]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
from xray_fov.nn.defaults import *
from xray_fov.nn.defaults.hyperparameters import hyperparameters as hp
from xray_fov.nn.defaults.loaders import loaders
from xray_fov.nn.defaults.models import model, optimizer
from xray_fov.nn.defaults.train import train
from xray_fov.paths import paths, outs
from xray_fov.data.dicom import dicom_directory
from xray_fov.nn.models.utils import create_model_from_hash, create_loaders_from_hash, load_model
import os

# setup

In [3]:
dicoms = dicom_directory(paths["dicom"]["phantom-alex"], caching=False)
reference_path = os.path.join(paths["dicom"]["phantom-alex"], dicoms[0]["file_path"])
hp.training.num_epochs=5
ldrs = loaders("thinned", paths["nn"], reference_path, hp.data)
mdl = model("monai", outs["models"], ldrs, hp)
opt = optimizer("adam", mdl, outs["models"], hp.optimizer)

from pprint import pprint
pprint(hp)
pprint(ldrs)
pprint(mdl)
pprint(opt)

HyperParameters(data=DataParameters(pin_memory=True,
                                    batch_size=8,
                                    num_workers=20),
                training=TrainingParameters(device='cuda', num_epochs=5),
                optimizer=OptimizerParameters(learning_rate=0.0001))
Loaders(train=<torch.utils.data.dataloader.DataLoader object at 0x7eeb85849be0>,
        test=<torch.utils.data.dataloader.DataLoader object at 0x7eeb857ead50>,
        validation=<torch.utils.data.dataloader.DataLoader object at 0x7eeb85875bd0>,
        data_dict={'data_dir': {'test-images': '/home/maximilianglumann/master-thesis/data/alex/segmentation/data/test/images',
                                'test-masks': '/home/maximilianglumann/master-thesis/data/alex/segmentation/data/test/masks',
                                'train-images': '/home/maximilianglumann/master-thesis/data/alex/segmentation/data/train/images',
                                'train-masks': '/home/maximiliangluman

# train

In [4]:
try:
    net = create_model_from_hash(mdl.hash, outs["models"]).to(hp.training.device)
    load_model(net, mdl.hash, outs["models"])
    train_loader, test_loader, validation_loader = create_loaders_from_hash(mdl.hash, outs["models"], batch_size=1)
except:
    train(outs["models"], hp.training, ldrs, mdl, opt)
    net = create_model_from_hash(mdl.hash, outs["models"]).to(hp.training.device)
    load_model(net, mdl.hash, outs["models"])
    train_loader, test_loader, validation_loader = create_loaders_from_hash(mdl.hash, outs["models"], batch_size=1)

100%|███████████████████████████████████████████████████████████████████████| 62/62 [00:41<00:00,  1.48it/s, loss=0.341]
/home/maximilianglumann/master-thesis/venvs/ma/lib/python3.14/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.metrics.utils get_mask_edges:always_return_as_numpy: Argument `always_return_as_numpy` has been deprecated since version 1.5.0. It will be removed in version 1.7.0. The option is removed and the return type will always be equal to the input type.
  warn_deprecated(argname, msg, warning_category)


Epoch: 5, Acc: 0.92, and Dice score: 0.75, IoU: 0.68, hd: 56.19
=> Saving optimizer
